# ICLR 2026 OpenReview data collection

Uses OpenReview **API v2** (api2.openreview.net). Single bulk fetch via invitation—no per-paper forum calls, so collection is fast (no rate-limit bottlenecks).

In [1]:
# Install if needed (uncomment)
# !pip install openreview-py "urllib3<2.0"

import openreview
import pandas as pd
from pathlib import Path

OUTPUT_DIR = Path("")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

In [2]:
def _flatten_content(content):
    """API v2 content uses {field: {value: ...}}. Extract values."""
    return {k: v.get("value", v) if isinstance(v, dict) else v for k, v in content.items()}


def get_iclr_2026_submissions():
    """Fetch all ICLR 2026 submissions in one bulk call via API v2."""
    client = openreview.api.OpenReviewClient(baseurl="https://api2.openreview.net")

    invitation_id = "ICLR.cc/2026/Conference/-/Submission"
    print(f"Collecting submissions from {invitation_id}...")

    submissions = client.get_all_notes(invitation=invitation_id)
    print(f"Successfully collected {len(submissions)} submissions.")

    for i, note in enumerate(submissions[:5]):
        title = note.content.get("title", {}).get("value", "No Title")
        print(f"  {i+1}. {title}")

    return submissions

In [3]:
submissions = get_iclr_2026_submissions()

df = pd.DataFrame([_flatten_content(n.content) for n in submissions])
out_path = OUTPUT_DIR / "iclr2026_submissions.csv"
df.to_csv(out_path, index=False)
print(f"Saved {len(df)} rows to {out_path}")

Retrying request: GET /notes?invitation=ICLR.cc%2F2026%2FConference%2F-%2FSubmission&stream=true, response: {'name': 'RateLimitError', 'message': 'Too many requests: You have made 62 requests, surpassing the limit of 60 requests. Please try again in 54 seconds (2026-02-15-6762492)', 'status': 429, 'details': {'limit': 60, 'remaining': 0, 'resetTime': '2026-02-15T18:33:34.063Z', 'used': 62, 'current': 62, 'reqId': '2026-02-15-6762492'}}, error: None
Successfully collected 19814 submissions.
  1. Improving Developer Emotion Classification via LLM-Based Augmentation
  2. Quantum-Inspired Image Encodings for Financial Time-Series Forecasting
  3. SAVIOR: Sample-efficient Alignment of Vision-Language Models for OCR Representation
  4. Revisiting Multilingual Data Mixtures in Language Model Pretraining
  5. One-Shot Style Personalization for RL Agents via Latent Discriminator
Saved 19814 rows to iclr2026_submissions.csv
